In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error as RMSE
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import mean_absolute_error as MAE
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor

In [2]:
df = pd.read_csv('data/comp-merged.csv')
dates = df['date']
df.drop(columns=['date', 'diff', 'bias_coef'], axis=1, inplace=True)
df.head()

,value_acc,value_met
0,0.000000,4.1
1,28.224999,16.0
2,44.069999,37.1
3,0.920000,5.1
4,24.969999,17.0


In [3]:
X = df.drop(columns=['value_met'], axis=1)
y = df['value_met']

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)#, random_state=42)

In [5]:
svr_params = {
    'C': [0.1, 1, 10, 100],
    'gamma': [1, 0.1, 0.01, 0.001],
    'kernel': ['rbf']
}

tuned_svr = GridSearchCV(SVR(), svr_params, cv=5, n_jobs=-1, verbose=1)
tuned_svr.fit(X_train, y_train)

print(tuned_svr.best_params_)
print(tuned_svr.best_score_)

Fitting 5 folds for each of 16 candidates, totalling 80 fits
{'C': 100, 'gamma': 0.001, 'kernel': 'rbf'}
0.16528528742647375


In [6]:
model = tuned_svr.best_estimator_
y_pred = model.predict(X_test)
print(f'RMSE: {RMSE(y_test, y_pred)}')
print(f'MSE: {MSE(y_test, y_pred)}')
print(f'MAE: {MAE(y_test, y_pred)}')

RMSE: 6.136392337464598
MSE: 37.65531091929424
MAE: 1.7342102881452555


In [7]:
xgb_params = {
    'n_estimators': [100, 500, 1000],
    'max_depth': [3, 5, 7, 9],
    'learning_rate': [0.01, 0.1, 0.3, 0.5]
}

tuned_xgb = GridSearchCV(xgb.XGBRegressor(), xgb_params, cv=5, n_jobs=-1, verbose=1)
tuned_xgb.fit(X_train, y_train)

print(tuned_xgb.best_params_)
print(tuned_xgb.best_score_)

Fitting 5 folds for each of 48 candidates, totalling 240 fits
{'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 100}
0.2153295524176458


In [8]:
model = tuned_xgb.best_estimator_
y_pred = model.predict(X_test)
print(f'RMSE: {RMSE(y_test, y_pred)}')
print(f'MSE: {MSE(y_test, y_pred)}')
print(f'MAE: {MAE(y_test, y_pred)}')

RMSE: 6.19376521034303
MSE: 38.36272748085564
MAE: 2.6867022957395297


In [9]:
rfr_params = {
    'n_estimators': [100, 500, 1000],
    'max_depth': [3, 5, 7, 9],
    'min_samples_split': [2, 5, 10],
    'random_state': [32]
}
tuned_rfr = GridSearchCV(RandomForestRegressor(), rfr_params, cv=5, n_jobs=-1, verbose=1)
tuned_rfr.fit(X_train, y_train)

print(tuned_rfr.best_params_)
print(tuned_rfr.best_score_)

Fitting 5 folds for each of 36 candidates, totalling 180 fits
{'max_depth': 3, 'min_samples_split': 10, 'n_estimators': 1000, 'random_state': 32}
0.20030179301779133


In [10]:
model = tuned_rfr.best_estimator_
y_pred = model.predict(X_test)
print(f'RMSE: {RMSE(y_test, y_pred)}')
print(f'MSE: {MSE(y_test, y_pred)}')
print(f'MAE: {MAE(y_test, y_pred)}')

RMSE: 5.836168769788786
MSE: 34.06086590945796
MAE: 2.1979282427074875


In [11]:
model = tuned_svr.best_estimator_
y_pred_test = model.predict(X_test)
y_pred_train = model.predict(X_train)
print(f'RMSE test: {RMSE(y_test, y_pred_test)}')
print(f'MSE test: {MSE(y_test, y_pred_test)}') 
print(f'MAE test: {MAE(y_test, y_pred_test)}')
print(f'RMSE train: {RMSE(y_train, y_pred_train)}')
print(f'MSE train: {MSE(y_train, y_pred_train)}')
print(f'MAE train: {MAE(y_train, y_pred_train)}')

RMSE test: 6.136392337464598
MSE test: 37.65531091929424
MAE test: 1.7342102881452555
RMSE train: 7.781369863923001
MSE train: 60.54971695916907
MAE train: 2.45613346722755


In [12]:
y_pred = np.concatenate((y_pred_train, y_pred_test))
y_true = np.concatenate((y_train, y_test))
print(f'RMSE all: {RMSE(y_true, y_pred)}')
print(f'MSE all: {MSE(y_true, y_pred)}')
print(f'MAE all: {MAE(y_true, y_pred)}')

RMSE all: 7.481083590699889
MSE all: 55.96661169103914
MAE all: 2.3116156352525885


In [13]:
df_corr = pd.read_csv('data/beirut-daily-precipitation.csv')
x = df_corr['value']
y_corr = model.predict(x.values.reshape(-1, 1))
df_corr['value'] = y_corr
df_corr.to_csv('./data/beirut-daily-corrected.csv', index=False)

/home/edargham/anaconda3/envs/thesis/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but SVR was fitted with feature names
  warnings.warn(


In [ ]:
pd.DataFrame({'date': dates, 'value_acc_new': y_pred, 'value_met': y_true}).to_csv('data/beirut-daily-comp.csv', index=False)